In [1]:
import ee
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

Conectarse a GEE

In [2]:
ee.Authenticate()
ee.Initialize(project='ee-apr160')
print(ee.String('Hello from the Earth Engine servers!').getInfo())


Hello from the Earth Engine servers!


In [9]:
year = 2024
fecha = datetime.strptime(f'{year}-01-01', '%Y-%m-%d')
fecha_inicio = fecha - timedelta(days=96)
fecha_fin = datetime.strptime(f'{year+1}-01-01', '%Y-%m-%d')

In [10]:
df_data = pd.read_csv(f'data/{year}.csv',encoding = "ISO-8859-1")


In [12]:
def add_date(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    
    return image.addBands(ee.Image(img_date).rename('date').toInt())

def save_coordinate(Sentinel_data,main_folder, row, feature_list,name_folder,src_data):
    features = []
    poi_geometry = ee.Geometry.Point([row['Longitud'], row['Latitud']])
    poi_properties = dict(row)
    poi_feature = ee.Feature(poi_geometry, poi_properties)
    features.append(poi_feature)
    ee_fc = ee.FeatureCollection(features) 


    def rasterExtraction(image):
        feature = image.sampleRegions(
            collection = ee_fc, # feature collection here
            scale = 10 # Cell size of raster
        )
        return feature
    
    results = Sentinel_data.filterBounds(ee_fc).select(feature_list).map(add_date).map(rasterExtraction).flatten()

    # You can modify this for better optimization
    column_df = list(src_data.columns)
    column_df.extend(feature_list + ['date'])
    
    nested_list = results.reduceColumns(ee.Reducer.toList(len(column_df)), column_df).values().get(0)
    data = nested_list.getInfo()
    df_final = pd.DataFrame(data,columns=column_df)
    df_final['date'] = pd.to_datetime(df_final['date'],format='%Y%m%d') 
    #df_final = df_final.groupby(["date","Latitud","Longitud"]).agg(
    #            Psurf_f_inst=('Psurf_f_inst', 'mean'),
    #            Qair_f_inst=('Qair_f_inst', 'mean'),
    #            Rainf_tavg=('Rainf_tavg', 'mean'),
    #            Tair_f_inst=('Rainf_tavg', 'mean'),
    #            Wind_f_inst=('Wind_f_inst', 'mean'),
    #            RootMoist_inst=('RootMoist_inst', 'mean')) \
    #            .reset_index()           

    df_final.to_csv(main_folder+name_folder+'/' + str(row['Latitud']) + '_' + str(row['Longitud']) + '.csv')#, mode='a', header=False)
   

def get_data(main_folder, source,from_date,to_date,feature,name_folder, src_data):
    Sentinel_data = ee.ImageCollection(source) \
    .filterDate(from_date,to_date) \
    .map(add_date)

    for index, row in src_data.iterrows():
        save_coordinate(Sentinel_data,main_folder, row,feature,name_folder,src_data)
        print('.',end='')

In [13]:
data = pd.read_csv(f'data/{year}-unique-points.csv')
data = data[['Latitud','Longitud']]

In [ ]:
get_data(f'gee-coordinates-{year}/', \
         'NASA/ORNL/DAYMET_V4', \
            fecha_inicio.strftime('%Y-%m-%d'), \
            fecha_fin.strftime('%Y-%m-%d'), \
            ['prcp','srad','tmax','tmin','vp'],'DAYMET_V4',data)

.............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [37]:
get_data(f'gee-coordinates-{year}/',
         'NASA/GLDAS/V021/NOAH/G025/T3H',
         "2024-01-01","2024-08-19",
         ['Psurf_f_inst','Qair_f_inst','Rainf_tavg','RootMoist_inst','Tair_f_inst','Wind_f_inst'],
         'SOIL',data)


....

In [41]:
get_data('gee-coordinates-prcp/',
         'IDAHO_EPSCOR/GRIDMET',
         "2021-09-20","2022-12-31",
         ['pr','sph','th','vs'],'IDAHO_EPSCOR',df_data)

....